# Sentiment Analysis using LSTM and CNN

**There are 10 points in total for this homework. Send the completed notebook to beroth@cis.uni-muenchen.de. The deadline is Tuesday, December 19, 23:59. You can work in teams of 2 or 3. This is the last exercise before the projects.**

First some imports. You will have to add imports for the CNN in the second part of the exercise.

In [1]:
import collections
import random
import nltk
from nltk.corpus import movie_reviews
from nltk.tokenize import word_tokenize
from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, LSTM, Bidirectional, Conv1D, GlobalMaxPooling1D, Input, concatenate
from keras.layers import add as addition
import sys
#TODO
random.seed(111)

Using TensorFlow backend.


The next part helps loading the sentiment data into the needed format. We will use the movie review corpus, which is provided by the nltk package. It classifies movie reviews as positive or negative. 

In [2]:
def nltk_data(n_texts_train=1500, n_texts_dev=500, vocab_size=10000):
    """
    Reads texts from the nltk movie_reviews corpus. A word2id dictionary is 
    created and the words in the texts are substituted with their numbers. Training
    and Development data is returned, together with labels and the word2id dictionary.
 
    :param n_texts_train: the number of reviews that will form the training data
    :param n_texts_dev: the number of reviews that will form the development data
    :param vocab_size: the maximum size of the vocabulary.

    :return list texts_train: A list containing lists of wordids corresponding to 
    training texts.
    :return list texts_dev: A list containing lists of wordids corresponding to 
    development texts.
    :return labels_train: A list containing the labels (0 or 1) for the corresponding
    text entry in texts_train
    :return labels_dev: A ilst containing the labels (0 or 1) for the corresponding
    text entry in texts_dev
    :return word2id: The dictionary obtained from the training texts that maps each
    seen word to an id.
    """
    all_ids = movie_reviews.fileids()
    if (n_texts_train+n_texts_dev>len(all_ids)):
        print ("Error: There are only",len(all_ids), "texts in the movie_reviews corpus. Training with all of those sentences.")
        n_texts_train=1500
        n_texts_dev=500
    posids = movie_reviews.fileids('pos')
    random.shuffle(all_ids)

    texts_train=[]
    labels_train=[]
    texts_dev=[]
    labels_dev=[]

    for i in range(n_texts_train):
        text = movie_reviews.raw(fileids=[all_ids[i]])
        tokens = [word.lower() for word in word_tokenize(text)]
        texts_train.append(tokens)
        if all_ids[i] in posids:       
            labels_train.append(1)
        else:
            labels_train.append(0)

    for i in range(n_texts_train, n_texts_train+n_texts_dev):
        text = movie_reviews.raw(fileids=[all_ids[i]])
        tokens = [word.lower() for word in word_tokenize(text)]
        texts_dev.append(tokens)
        if all_ids[i] in posids:
            labels_dev.append(1)
        else:
            labels_dev.append(0)

    word2id=create_dictionary(texts_train, vocab_size)
    texts_train = [to_ids(s,word2id) for s in texts_train]
    texts_dev = [to_ids(s,word2id) for s in texts_dev]
    return (texts_train, labels_train, texts_dev, labels_dev, word2id)

def create_dictionary(texts, vocab_size):
    """
    Creates a dictionary that maps words to ids. More frequent words have lower ids.
    The dictionary contains at the vocab_size-1 most frequent words (and a placeholder '<unk>' for unknown words).
    The place holder has the id 0.
    """
    counter = collections.Counter()
    for tokens in texts:
        counter.update(tokens)
    vocab = [w for w,c in counter.most_common(vocab_size-1)]
    word_to_id = {w:(i+1) for i,w in enumerate(vocab)} # TODO
    word_to_id[UNKNOWN_TOKEN] = 0 # TODO
    return word_to_id # TODO

def to_ids(words, dictionary):
    """
    Takes a list of words and converts them to ids using the word2id dictionary.
    """
    # TODO
    ids=[]
    for word in words:
        ids.append(dictionary.get(word, dictionary[UNKNOWN_TOKEN]))
    return ids


We define a couple of constants, which should be familiar from the last exercise and fetch the data. You don't have to remove the download part, it will check automatically if it is already downloaded.

In [4]:
VOCAB_SIZE = 10000
MAX_LEN = 100
BATCH_SIZE = 32
EMBEDDING_SIZE = 20
HIDDEN_SIZE = 10
EPOCHS = 10
UNKNOWN_TOKEN = "<unk>"

nltk.download('movie_reviews')
nltk.download('punkt')
x_train, y_train, x_dev, y_dev, word2id = nltk_data(vocab_size=VOCAB_SIZE)
x_train = sequence.pad_sequences(x_train, maxlen=MAX_LEN)
x_dev = sequence.pad_sequences(x_dev, maxlen=MAX_LEN)
print(x_train[0])

[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/shs/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package punkt to /home/shs/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[  24  278   12   10    0    1   22  184   37  747    3   22  100  140   25
   18 5673    0   20   30 1065    1   24   54   28  274    7 2513   35  103
 1843    3    9  480    1   22  782   16   62   22  123 4147    7    4 1636
  161    4    0  405   21   17 2469    0   18    0    1    4    0 5142 1341
  121   21    1    5    4  752 2921  230   15    2  114    1    6  227   14
    3   22   75   28 1254   20   30    7  321    1 2410    1  409  104 8322
    3 5403   15 1784   14 1313  806    0   11 3009]


The first of three models that we will define is a Bidirectional LSTM. In comparison to the last exercise when you had to get an output at each timestep, this time you will only need the last output.

** TODO: Build model with bidirectional LSTM, with HIDDEN_SIZE size for each direction. After the LSTM, insert one additional dense layer (HIDDEN_SIZE and tanh non-linearity), before the label is predicted by the final layer.(1 p.) **

In [5]:
lstm_model = Sequential()
lstm_model.add(Embedding(VOCAB_SIZE, EMBEDDING_SIZE))
lstm_model.add(Bidirectional(LSTM(HIDDEN_SIZE))) # TODO
lstm_model.add(Dense(HIDDEN_SIZE, activation='tanh')) # TODO
lstm_model.add(Dense(1, activation='sigmoid')) # TODO

Train and evaluate the model

In [7]:
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(x_dev, y_dev))
score, acc = lstm_model.evaluate(x_dev, y_dev)

Train on 1500 samples, validate on 500 samples
Epoch 1/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0048 - acc: 1.0000 - val_loss: 0.9777 - val_acc: 0.7480
Epoch 2/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0021 - acc: 1.0000 - val_loss: 1.0877 - val_acc: 0.7360
Epoch 3/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0012 - acc: 1.0000 - val_loss: 1.1416 - val_acc: 0.7420
Epoch 4/10
1500/1500 [==============================] - 4s 3ms/step - loss: 9.0902e-04 - acc: 1.0000 - val_loss: 1.1914 - val_acc: 0.7440
Epoch 5/10
1500/1500 [==============================] - 4s 3ms/step - loss: 7.1842e-04 - acc: 1.0000 - val_loss: 1.2382 - val_acc: 0.7480
Epoch 6/10
1500/1500 [==============================] - 4s 3ms/step - loss: 5.8663e-04 - acc: 1.0000 - val_loss: 1.2859 - val_acc: 0.7460
Epoch 7/10
1500/1500 [==============================] - 4s 3ms/step - loss: 4.9049e-04 - acc: 1.0000 - val_loss: 1.3278 - val_acc: 0.7420

In [8]:
print("LSTM Accuracy: ",acc)

LSTM Accuracy:  0.742


** TODO: Solve the same problem using a CNN+pooling over three-grams with 2\*HIDDEN_SIZE filters, and apply the tanh non-linearity. After CNN+pooling, insert as before one additional dense layer (HIDDEN_SIZE and tanh non-linearity), before the label is predicted by the final layer. (3 p.)**

In [9]:
cnn_model = Sequential() # NOTE: rnn_model vs cnn_model?
cnn_model.add(Embedding(VOCAB_SIZE, EMBEDDING_SIZE))
cnn_model.add(Conv1D(2*HIDDEN_SIZE,3,activation='tanh',use_bias=False)) # TODO
cnn_model.add(GlobalMaxPooling1D()) # TODO
#cnn_model.add(Dense(50, activation='relu')) # TODO
cnn_model.add(Dense(HIDDEN_SIZE, activation='tanh')) # TODO
cnn_model.add(Dense(1, activation='sigmoid')) # TODO
cnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 20)          200000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 20)          1200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 20)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 201,421
Trainable params: 201,421
Non-trainable params: 0
_________________________________________________________________


Train and evaluate

In [10]:
cnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
cnn_model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(x_dev, y_dev))
score, acc = cnn_model.evaluate(x_dev, y_dev)

Train on 1500 samples, validate on 500 samples
Epoch 1/10
1500/1500 [==============================] - 1s 799us/step - loss: 0.6944 - acc: 0.5073 - val_loss: 0.6921 - val_acc: 0.4900
Epoch 2/10
1500/1500 [==============================] - 1s 456us/step - loss: 0.6709 - acc: 0.7820 - val_loss: 0.6878 - val_acc: 0.5340
Epoch 3/10
1500/1500 [==============================] - 1s 449us/step - loss: 0.6324 - acc: 0.9160 - val_loss: 0.6747 - val_acc: 0.5960
Epoch 4/10
1500/1500 [==============================] - 1s 448us/step - loss: 0.5490 - acc: 0.9493 - val_loss: 0.6385 - val_acc: 0.6700
Epoch 5/10
1500/1500 [==============================] - 1s 445us/step - loss: 0.4020 - acc: 0.9673 - val_loss: 0.6053 - val_acc: 0.6840
Epoch 6/10
1500/1500 [==============================] - 1s 439us/step - loss: 0.2337 - acc: 0.9920 - val_loss: 0.5734 - val_acc: 0.7200
Epoch 7/10
1500/1500 [==============================] - 1s 447us/step - loss: 0.1176 - acc: 0.9973 - val_loss: 0.5888 - val_acc: 0.6920
E

In [11]:
print("CNN Accuracy: ", acc)

CNN Accuracy:  0.711999999523


** TODO: How many parameters in total does the entire CNN model (all layers including embedding) have to optimize/learn? Show your calculation. (1.5 p.) **

** TODO: Use the Keras functional API to merge (combine) the LSTM sentence representation with the pooled representation. Instead of concatenation use vector addition for merging. The same embedding should be learned and used going into the LSTM and the CNN. As before, insert one additional dense layer (HIDDEN_SIZE and tanh non-linearity), before the label is predicted by the final layer. Train and evaluate as before. (5.5 p.)**
functional API: https://keras.io/getting-started/functional-api-guide/

In [12]:
input_seq = Input(shape=(MAX_LEN,)) # TODO 0.5
emb = Embedding(VOCAB_SIZE, EMBEDDING_SIZE)(input_seq) # TODO 0.5
lstm_vec = Bidirectional(LSTM(HIDDEN_SIZE))(emb) # TODO 0.5
cnn = Conv1D(2*HIDDEN_SIZE,3,activation='tanh')(emb) # TODO 0.5 
pool_vec = GlobalMaxPooling1D()(cnn) # TODO 0.5
combined_vec = addition([pool_vec,lstm_vec]) # TODO 0.5
final_vec = Dense(HIDDEN_SIZE, activation='tanh')(combined_vec) # TODO 0.5
prediction = Dense(1, activation='sigmoid')(final_vec) # TODO 0.5
model = Model(inputs=input_seq, outputs=prediction) # TODO 0.5
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) # TODO 0.33
model.summary()
model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(x_dev, y_dev))# TODO 0.33
score, acc = model.evaluate(x_dev, y_dev)# TODO 0.33

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 100, 20)      200000      input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 98, 20)       1220        embedding_3[0][0]                
__________________________________________________________________________________________________
global_max_pooling1d_2 (GlobalM (None, 20)           0           conv1d_2[0][0]                   
__________________________________________________________________________________________________
bidirectio

In [13]:
print("Compostion Accuracy: ", acc)

Compostion Accuracy:  0.695999999046
